In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.regularizers import L2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = 224

# Train data with validation split with data augmentation
train_generator = ImageDataGenerator(rescale=1./255, validation_split=0.25, 
                                        horizontal_flip=True, vertical_flip=True,
                                        rotation_range=90, zoom_range=0.2)

train= train_generator.flow_from_directory('data/cropped/mass/train/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=16, class_mode='binary', subset='training')
val= train_generator.flow_from_directory('data/cropped/mass/train/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=16, class_mode='binary', subset='validation')

In [ ]:
# Resnet 50 model
resnet = ResNet50(include_top=False, weights='imagenet', pooling="max")

model_layers = [
    resnet,
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid'),
]

model = Sequential(model_layers)

In [ ]:
# Get image of model
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


In [ ]:
optimizer = Adam(0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(train, validation_data=val, epochs=30, callbacks=callbacks_list)

In [ ]:
# Plot the training and validation accuracy
acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

In [ ]:
# Plot the loss
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

plt.plot(epochs, loss, 'r', label='Loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

In [ ]:
# save model
model.save('model.h5')